# Vorlesung: Arbeiten mit Webseiten als Quellen

Viele Datenbank verfügen über eine Webseite, die eine Textsuche erlaubt. Das Result einer solchen Suche kann über das Python Paket beautifulsoup ausgelesen und in einen Dataframe geschrieben werden. 
Aufgrund dieser Daten kann dann weiter gearbeitet werden, sei es mit regulären Ausdrücken oder NLTK. 

In [346]:
import json
import re
import pandas as pd

import requests
from bs4 import BeautifulSoup

## Stellen einer Suchanfrage: Jordanus

Als erstes muss in der Suchmaske der Webseite eine Suche gestartet werden. Einige Datenbanken bieten auch eine sogenannte API an. 
Diese kann dazu genutzt werden, direkt ohne Umweg über ein Browser-Tab eine Suchanfrage zu stellen.

In diesem Beispiel ergibt die Suchanfrage einen Webseiten String, der einer Datenbank-Abfrage entspricht. Nun kann über das requests Paket eine Abfrage der Webseite gestartet werden und
das Resultat wird mittels BeautifulSoup in eine "Textsuppe" umgewandelt.

In [2]:
url = 'http://jordanus.badw.de/cgi-bin/lrz_jord-search.pl?sprache=en&datenbank=ptolemaeus&ausgabe=dhstext&ausdateiformat=txt&listpos=0&listen=keine&listlet=keiner&fn=t52a56.f&fi=Martianus+Capella'
    
r  = requests.get(url)

data = r.text

soup = BeautifulSoup(data,'lxml')

## Beautiful Soup Basics



Get the title of the html page

In [3]:
soup.title

<title>Search result</title>

Find all paragraphs

In [4]:
soup.findAll('p')

[<p>
 </p>]

Find all links

In [5]:
soup.findAll('a')[:4]

[<a href="http://jordanus.badw.de/cgi-bin/lrz_jord-form.pl?sprache=en"><font color="#8A2020" size="+1">New search</font>
 </a>,
 <a href="http://jordanus.badw.de/cgi-bin/lrz_jord-search.pl?sprache=en&amp;datenbank=ptolemaeus&amp;ausgabe=dhs&amp;ausdateiformat=txt&amp;listpos=0&amp;listen=keine&amp;listlet=keiner&amp;fn=t52a56.f&amp;fi=Martianus+Capella"><b>ACAPR00001/48</b></a>,
 <a href="http://jordanus.badw.de/cgi-bin/lrz_jord-search.pl?sprache=en&amp;datenbank=ptolemaeus&amp;ausgabe=dhs&amp;ausdateiformat=txt&amp;listpos=0&amp;listen=keine&amp;listlet=keiner&amp;fn=t52a56.f&amp;fi=Martianus+Capella"><b>ANEYHC3970726/02</b></a>,
 <a href="http://jordanus.badw.de/cgi-bin/lrz_jord-search.pl?sprache=en&amp;datenbank=ptolemaeus&amp;ausgabe=dhs&amp;ausdateiformat=txt&amp;listpos=0&amp;listen=keine&amp;listlet=keiner&amp;fn=t52a56.f&amp;fi=Martianus+Capella"><b>BBRGS0424/04</b></a>]

Die so erhaltenen Daten können nun je nach Struktur der Webseite nach Merkmalen durchsucht werden. Bei der Jordanus Webseite (http://jordanus.badw.de/) werden die Daten als Tabelle dargestellt. 
Deshalb wird nun nach dem Wort "table" gesucht. Jede Tabelle ist mit Zeilen ("tr") und Spalten ("td") strukturiert. Der Inhalt jeder Zeile wird in ein Schlüssel:Wert Paar umgewandelt und in eine Liste geschrieben.   

In [6]:
data = []
tables = soup.find_all('table')

for table in tables:
    item = {}
    table_body = table.find_all('tr')
    for row in table_body:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if cols:
            item[cols[0]] = cols[1]
    data.append(item) 

In [7]:
data[:4]

[{'Search expr.:': 'Author = Martianus Capella'},
 {'Number of records found: \n82': 'New search'},
 {},
 {'2. Incipit': 'Item Machrobius. Luna item circuli',
  '2/48': 'ACAPR00001/48',
  'Add. numbers': 'Gen. Sci. 1',
  'Author standard': 'Macrobius\nMartianus Capella',
  'Century': '12\n13\n14\n15',
  'City': 'San Juan Capistrano',
  'Condition': '(152-154): these leaves are slit and rather crudely repaired. Apart from these three leaves the whole of this manuscript is in fine condition\nUse of a piece of parchment from an English language bill of conveyance to repair (150)',
  'Contents': 'Comm. 1.20.14 (p. 81, line 10) to 1.20.32 (p.84, line 27); extract from Felix Capella; (80r) Macrobius. Eratostenes philosophus ideoque geometra (...)',
  'Country': 'USA',
  'Cover': 'Bound in 15th century calf\nBound in old English calf',
  'Decorations': '(152v-154v) Colour drawings of constellations\n(152-154) contain 30 remarkable English pen-drawings of the early 13th century; initials mainl

## Erzeugung des Dataframes

Der Dataframe kann nun einfach erstellt werden. In einem ersten Schritt sind aber noch zu viele Schlüssel enthalten, die keine auswertbare Information ergeben. 

In [8]:
df = pd.DataFrame(data)
print(df.shape)
df.keys()

(85, 182)


Index(['10/3', '11/2', '12/3', '12/4', '13/2', '13/4', '13/5', '13/6', '14/3',
       '15/9',
       ...
       'Source', 'Source's source', 'Stype of writing', 'Subject 1',
       'Subject 2', 'T & K', 'Type of ms. 2', 'Type of source', 'Watermarks',
       'Width of columns'],
      dtype='object', length=182)

Um eine Untermenge des Dataframes auszuwählen, können wir reguläre Ausdrücke nutzen. Alle Schlüssel der Form "[Zahlen]/[Zahlen]" sollen nicht angezeigt werden. Einge neue Liste mit sinnvollen Schlüsseln wird erzeugt und als Auswahlkriterium an den Dataframe übergeben.  

In [9]:
keyList = [x for x in df.keys() if not re.findall('\d+/\d+',x)]

In [10]:
dfReducedCapella= df[keyList].dropna(axis=1,how='all')
dfReducedCapella.shape

(85, 75)

In [11]:
dfReducedCapella.head(5)

,2. Incipit,Add. numbers,Addit. Names,Addit. copies,Addit. inform.,Addit. remarks,Additional Texts,Additional parts,Author standard,Autor ms.,...,Source,Source's source,Stype of writing,Subject 1,Subject 2,T & K,Type of ms. 2,Type of source,Watermarks,Width of columns
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Item Machrobius. Luna item circuli,Gen. Sci. 1,NaN,NaN,NaN,NaN,NaN,NaN,Macrobius\nMartianus Capella,NaN,...,Faye/Bond (1962)\nAuktionskatalog Robinson\nN....,NaN,"(1-128) written in a very fine book-hand, the ...",NaN,NaN,TK 699,NaN,cat.\ncat.\nts. pvt. notes\nts. pvt. notes,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Liber VIII: De astronomia,NaN,NaN,Martianus Capella\nMacrobius (attrib.),NaN,...,Faulhaber (1983),NaN,Minute gothic minuscule hand,NaN,NaN,TK 1183,NaN,cat.,"w : (inner ff.II., older than outer ones) crow...","166 x 86, first line below top ruling"


Die gewünschten Schlüssel können auch direkt als Liste übergeben werden und so eine noch reduziertere Form des Dataframes erzeugen.

In [12]:
smallList = ['Author standard','Short title','Short title st.','Library','Language','Folio no.','City','Country','Century']
df[smallList].dropna(how='all').reset_index(drop=True).head(5)

,Author standard,Short title,Short title st.,Library,Language,Folio no.,City,Country,Century
0,Macrobius\nMartianus Capella,NaN,Selections from Macrobius (part.),The Library of Robert B. Honeyman Jr.,Latin,79r-80v,San Juan Capistrano,USA,12\n13\n14\n15
1,Martianus Capella\nMacrobius (attrib.),NaN,De nuptiis Philologiae et Mercurii (part.),Library of the Hispanic Society of America,Latin,1r-6v,New York,USA,NaN
2,Martianus Capella,NaN,Annotations de genre divers,Stedelijke Openbare Bibliotheek,Latin,309r-311r,Brugge,Belgium,14
3,Martianus Capella,Recueil de traités scientifiques,De nuptiis Philologiae et Mercurii,Bibliothèque Royale,Latin,13v-196v,Bruxelles,Belgium,10
4,Martianus Capella,NaN,"De nuptiis Philologiae et Mercurii (part., inc.)",Staatsbibliothek,Latin,65r-79r,Bamberg,Germany,10


## Suchanfrage II: isisCB explore

Die Webseite für das isisCB explore Tool der "Isis Bibliography of the History of Science" ist strukturell anders. Hier werden Suchergebnisse über eine Link-Liste dargestellt. 
In der Liste werden einzelne Ergebnisse in paragraphen sortiert (mit dem tag `<p>`) . Wir generieren eine Unterliste aller paragraphen die das Word `Article` oder `Book` enthalten. 

In [13]:
url2 = 'https://data.isiscb.org/isis/?q=copernicus&models=isisdata.citation&sort_order_citation=publication_date_for_sort&sort_order_dir_citation=descend&sort_order_dir_authority=ascend&selected_facets=citation_subject_ids_exact:CBA000021037'

r2  = requests.get(url2)

data2 = r2.text

soup2 = BeautifulSoup(data2,'lxml')

In [14]:
baseURL = 'https://data.isiscb.org/'

Suche alle Paragraphen, die im Text das Wort Article oder Book enthalten.

In [15]:
plist = soup2.findAll('p')
queListe = [x for x in plist if "Article" in x.text or "Book" in x.text]

Für alle gefundenen Datensätze müssen wir nun einem weiteren Link folgen. Dieser wird über den tag `<a>` gesucht, mit der Option `href=True`.
Mit dem requests Paket wird eine Anfrage `GET` an die URL gesendet, die sich aus der Basis-URL und dem gefundenen Link zusammensetzt. 

Das Resultat wird in eine Soup verwandelt. In der neuen Webseite werden die Informationen zu einem Artikel oder Buch über das Element `div` mit der Klasse `class=col-sm 5` dargestellt. 
In einem solchen Element stehen die Informationen wieder in Paragraphen, die in eine Untermenge eingetragen werden.

In [16]:
resultList = []
for x in queListe:
    links = x.findAll('a',href=True)
    for link in links:
        scndLevel = baseURL + link['href']
        scndRes = requests.get(scndLevel)
        dataTemp = scndRes.text
        soupTemp = BeautifulSoup(dataTemp,'lxml')
        x = soupTemp.findAll('div',class_='col-sm-5')
        pListTemp = x[0].findAll('p')
        subList = []
        for parag in pListTemp:
            res = parag.text
            subList.append(res)
        resultList.append(subList)    

Mittels regex wird aus der Menge aller Resultate eine Menge von Dictionaries gebaut, vorraus wir einen sortierten Dataframe erhalten.

In [17]:
dictList = []
for i in range(len(resultList)):
    subDict = {}
    for k in range(len(resultList[i])):
        keys = re.findall('.+?(?=:)',resultList[i][k])
        value =re.findall('(?<=:).+',resultList[i][k]) 
        if keys:
            if value:
                subDict[keys[0]] = value[0]#resultList[i][k]
    dictList.append(subDict)

In [18]:
dfISISTemp = pd.DataFrame(dictList)
dfISISTemp.head(5)

,http,Adventures in the Bone Trade,Lee De Forest,Secrets of the Old One,Abstract,Authors & Contributors,Description,Publication Date
0,//data.isiscb.org/isis/citation/CBB001551118/,NaN,NaN,NaN,The contributors to Making of Copernicus exam...,"Neuber, Wolfgang (Author) ; Rahn, Thomas (Aut...",NaN,2015
1,//data.isiscb.org/isis/citation/CBB001551912/,NaN,NaN,NaN,At the center of this article is an iconograp...,"Gaulke, Karsten (Author) ;",NaN,2015
2,//data.isiscb.org/isis/citation/CBB001451084/,NaN,NaN,NaN,Copernicus makes his dislike of the equant kn...,"Blåsjö, Viktor (Author) ;",NaN,2014
3,//data.isiscb.org/isis/citation/CBB001510107/,NaN,NaN,NaN,Is science more rational or objective than an...,"Parsons, Keith M. (Author) ;",NaN,2014
4,//data.isiscb.org/isis/citation/CBB001420393/,NaN,NaN,NaN,During the two decades before the turning poi...,"Blumenthal, Geoffrey (Author) ;",NaN,2014


Um den Dataframe zu bereinigen, verwerfen wir alle Spalten in denen weniger als zwei Werte ungleich NaN sind. Zudem verwerfen wir alle Zeilen, die nur NaN Einträge enthalten.

In [19]:
dfISIS = dfISISTemp.dropna(axis=1,thresh=2)
dfISIS = dfISIS.dropna(axis=0,how='all')
dfISIS

,http,Abstract,Authors & Contributors,Publication Date
0,//data.isiscb.org/isis/citation/CBB001551118/,The contributors to Making of Copernicus exam...,"Neuber, Wolfgang (Author) ; Rahn, Thomas (Aut...",2015
1,//data.isiscb.org/isis/citation/CBB001551912/,At the center of this article is an iconograp...,"Gaulke, Karsten (Author) ;",2015
2,//data.isiscb.org/isis/citation/CBB001451084/,Copernicus makes his dislike of the equant kn...,"Blåsjö, Viktor (Author) ;",2014
3,//data.isiscb.org/isis/citation/CBB001510107/,Is science more rational or objective than an...,"Parsons, Keith M. (Author) ;",2014
4,//data.isiscb.org/isis/citation/CBB001420393/,During the two decades before the turning poi...,"Blumenthal, Geoffrey (Author) ;",2014
5,//data.isiscb.org/isis/citation/CBB001422065/,"With exoplanets being discovered daily, Earth...","Gingerich, Owen (Author) ;",2014
6,//data.isiscb.org/isis/citation/CBB001321204/,"This essay studies Moses Galeano, a Jewish sc...","Morrison, Robert (Author) ;",2014
7,//data.isiscb.org/isis/citation/CBB001500389/,William Shakespeare lived at a remarkable tim...,"Falk, Dan (Author) ;",2014
8,//data.isiscb.org/isis/citation/CBB001510030/,This volume brings John Milton's Paradise Los...,"Danielson, Dennis Richard (Author) ;",2014
9,//data.isiscb.org/isis/citation/CBB001213931/,In Copernicus in the Cultural Debates of the ...,"Omodeo, Pietro Daniel (Author) ;",2014


## Suchanfrage III: Newtons Gesammelte Werke 

Die Homepage des Newton Projects der University of Sussex, http://www.newtonproject.sussex.ac.uk/, bietet besser strukturierte Inhalte. 
Hier können die Informationen mit passenden Klassennamen sinnvoll ausgelesen werden. 

Die genauen Spezifikation sind als PDF verfügbar: http://www.newtonproject.sussex.ac.uk/resources/pdfs/techspec.pdf

In [3]:
url3 = 'http://www.newtonproject.sussex.ac.uk/prism.php?id=43'

r3  = requests.get(url3)

data3 = r3.text

soup3 = BeautifulSoup(data3,'lxml')

In [4]:
baseURLNewton = 'http://www.newtonproject.sussex.ac.uk'

Die Einträge werden in einer Table mit der Klasse `record` ausgegeben. 

In [5]:
allrecords = soup3.findAll('td',class_='record')

Der Title eines Eintrags:

In [6]:
allrecords[0].findAll('p',class_='title')[0].text

"Fair copies of the 'Short Chronicle' and 'Chronology of Ancient Kingdoms Amended'"

den Author erhalten wir über 

In [7]:
allrecords[0].findAll('p',class_='author')[0].text

'Author: Isaac Newton'

ebenso verfahren wir mit metadata, source, links und primary_key. Wieder wird eine Liste mit Sub-Dictionaries erzeugt und daraus ein Dataframe gebaut. 

Um den normalisierten Volltext zu erhalten, folgen wir wieder einem Link auf die Unterseite und extrahieren dort den Text. Dieser ist eindeutig markiert mit der id `tei`
Über den Befehl `.text` erhalten wir den Text ohne die HTML-Struktur.

**Achtung:**

Das Abfragen von allen Einträgen dauert etwas länger. Daher werden unten nur die ersten 10 Einträge abgefragt (`[:10]`) um den Server der Uni Sussex nicht zu überlasten. Wenn die Texte mit NLTK weiter bearbeitet werden sollen, bietet es sich daher an, den resultierenden Dataframe als Pickle abzuspeichern. 

In [332]:
dictListNewton = []
for record in soup3.findAll('td',class_='record')[:10]:
    subDict = {}
    for keyClass in ['title', 'author','metadata','source','primary_key']:
        try:
            value = record.findAll('p',class_=keyClass)[0].text
            subDict[keyClass] = value
        except:
            subDict[keyClass] = None
    links = record.findAll('a',href=True,text='Normalized\xa0Text')
    if links:
        scndLevel = baseURLNewton + links[0]['href']
        scndRes = requests.get(scndLevel)
        dataTemp = scndRes.text
        soupTemp = BeautifulSoup(dataTemp,'lxml')
        text = soupTemp.findAll('div',id='tei')
        subDict['norm_text'] = text[0]
    dictListNewton.append(subDict)    

In [333]:
dfNewton = pd.DataFrame(dictListNewton)
dfNewton.head(3)

,author,metadata,norm_text,primary_key,source,title
0,Author: Isaac Newton,"Metadata: in English, c. 86,051 words, 120 ff.","<div id=""tei""> <div> <span class=""pagenumber"" ...",Newton Catalogue ID: THEM00090,"Source: MS Add. 3988, Cambridge University Lib...",Fair copies of the 'Short Chronicle' and 'Chro...
1,Author: Isaac Newton,"Metadata: in Latin, c. 6,207 words, 21 pp. on ...","<div id=""tei""> <div> <span class=""pagenumber"" ...",Newton Catalogue ID: THEM00046,"Source: Yahuda Ms. 3, National Library of Isra...",'Introductio. Continens Apocalypseos rationem ...
2,Author: Isaac Newton,"Metadata: mainly in Latin with some English, G...","<div id=""tei""> <div> <span class=""pagenumber"" ...",Newton Catalogue ID: THEM00056,"Source: Yahuda Ms. 13, National Library of Isr...",Miscellaneous theological extracts and notes


In [334]:
dfNewton.shape

(250, 6)

### Speichern und Laden von Dataframes

Die Struktur von Dataframes bietet es an, diese als JSON zu speichern. Das erfolgt einfach mit df.to_json('Dateiname'). Über die normale Einleseroutine von JSONs kann dann der Dataframe aus der Datei wieder hergestellt werden. Die bereits vorhandene JSON Datei enthält alle 250 Einträge zu Newton.

In [335]:
# Save dataframe to json file on disk
#dfNewton.to_json('./newton_metadata_frame.json')

In [329]:
# Load by using
# with open('./newton_metadata_frame.json') as NewtonData:
#     newtonJson = json.load(NewtonData)
#     dfTest = pd.DataFrame(newtonJson)

### Weitere Struktur: Paragraphen je Kapitel je Text

Um eine weitergehende Textanalyse durchzuführen, bietet es sich an, die Texte nach Paragraphen zu strukturieren und dann darauf Anlysemethoden anzuwenden.

In [336]:
chpStructure = []
for i in range(len(dfNewton)):
    title = str(i) + ': ' +  dfNewton['title'].iloc[i]
    try:
        #Konstruiere Liste aller Kapitel
        chps = dfNewton['norm_text'].iloc[i].findAll('div')
    except:
        pass
    if chps:
        for chp in range(len(chps)):
            pgsTemp = {}
            chKey = str(chp)
            # Konstruiere Liste aller Absätze
            paras = chps[chp].findAll('p')
            for pg in range(len(paras)):
                #print('Chp:{0},Para:{1}'.format(chp,pg))
                keyTemp = str(pg)
                valueTemp = paras[pg].text
                # Speichere den Text des Paragraphen mit Buch,Kapitel, und Paragraphennummer als tuple
                triple = (title,chp,pg,valueTemp)
                chpStructure.append(triple)

In [345]:
dfNewtonParagraphs = pd.DataFrame.from_dict(chpStructure)
dfNewtonParagraphs.set_index(0, inplace=True) # Setze Text-Titel als Index
dfNewtonParagraphs.set_index(1, append=True, inplace=True) # Ergänze Kaptitel als zweite Ebene Index
dfNewtonParagraphs.index.rename(['title','chapter'], inplace=True)
dfNewtonParagraphs.columns = ['paragraph','paragraph_text'] # Benenne Spalten um
print('Anzahl der Paragraphen aller Newton Texte: {}'.format(dfNewtonParagraphs.shape[0]))
dfNewtonParagraphs.head()

Anzahl der Paragraphen aller Newton Texte: 33170


paragraph  \
title                                              chapter              
0: Fair copies of the 'Short Chronicle' and 'Ch... 0                0   
                                                   0                1   
                                                   1                0   
                                                   1                1   
                                                   1                2   

                                                                                               paragraph_text  
title                                              chapter                                                     
0: Fair copies of the 'Short Chronicle' and 'Ch... 0        This is the original manuscript from which Sir...  
                                                   0                                           Iohn Conduitt.  
                                                   1        The Greek Antiquities are full of poetical fic...  
                                                   1        So then a little after the death of Alexander ...  
                                                   1        But how uncertain their Chronology is, & how d...

In [351]:
#dfNewtonParagraphs.reset_index().to_json('./newton_paragraphs_frame.json')

In [349]:
# Load by using
#with open('./newton_paragraphs_frame.json') as NewtonParaData:
#    newtonParaJson = json.load(NewtonParaData)
#    dfTestPara = pd.DataFrame(newtonParaJson)
#dfTestPara

# Aus externeren JSON Daten

Auch mit Dritt-Programmen erstellte JSON Dateien können die Grundlage für eine Websuche bilden. Eine Liste mit 1000 Publikation zur Kalten Fusion dient hier als Beispiel.

In [378]:
with open('./PoPCites.json','r',encoding='utf-8-sig') as coldFusionData:
    coldFusionJson = json.load(coldFusionData)

In [382]:
dfColdFusion = pd.DataFrame(coldFusionJson)

In [385]:
dfColdFusion.keys()

Index(['article_url', 'authors', 'cites', 'cites_url', 'ecc', 'publisher',
       'rank', 'source', 'title', 'type', 'use', 'year'],
      dtype='object')

In [388]:
dfColdFusion.head(4)

,article_url,authors,cites,cites_url,ecc,publisher,rank,source,title,type,use,year
0,http://www.territorioscuola.com/download/fusio...,"[SE Jones, EP Palmer, JB Czirr, DL Decker, GL ...",746.0,https://scholar.google.com/scholar?cites=45967...,746.0,territorioscuola.com,1,Nature,Observation of cold nuclear fusion in condense...,PDF,True,1989
1,http://iopscience.iop.org/article/10.1143/JJAP...,"[R Taniguchi, T Yamamoto, S Irie]",51.0,https://scholar.google.com/scholar?cites=10046...,51.0,iopscience.iop.org,2,Japanese Journal of Applied …,Detection of charged particles emitted by elec...,NaN,True,1989
2,http://www.ans.org/pubs/journals/fst/a_29233,"[PK Iyengar, M Srinivasan, SK Sikka, A Shyam, ...",33.0,https://scholar.google.com/scholar?cites=13691...,33.0,ans.org,3,Fusion Science and …,Bhabha Atomic Research Centre studies in cold ...,NaN,True,1990
3,http://journals.aps.org/prl/abstract/10.1103/P...,"[AJ Leggett, G Baym]",82.0,https://scholar.google.com/scholar?cites=18390...,82.0,APS,4,Physical review letters,Exact upper bound on barrier penetration proba...,NaN,True,1989


Alle Artikel verfügen über eine URL Angabe, die aber teilweise auf nicht mehr funktionierende Seiten führt. 

Große Verlage, wie zB. Physical Review Letters, blockieren zudem die automatisierte PDF-Suche durch Captchas. 
Für wissenschaftliche Arbeiten kann teilweise ein extra Zugang zu Daten beantragt werden.

In [387]:
dfColdFusion['article_url'].iloc[5]

'http://journals.aps.org/prl/abstract/10.1103/PhysRevLett.62.2929'

# Geographische Daten

Antike Orte in dem Projekt https://pleiades.stoa.org/ können über eine API abgefragt werden: http://api.pleiades.stoa.org/

Zeige den aktuellen Zustand des Projekts

In [418]:
res = requests.get('http://api.pleiades.stoa.org/status')
res.json()

{'num_locations': 39363, 'num_names': 31012, 'num_places': 35204}

Erhalte Informationen zu einer bestimmten Stadt, kodiert durch ID.

In [419]:
res2 = requests.get('http://pleiades.stoa.org/places/423025/json')
jsonRes = res2.json()

In [420]:
jsonRes.keys()

dict_keys(['names', 'features', 'uri', '@type', 'id', 'description', 'rights', 'type', 'references', 'details', 'provenance', 'title', 'placeTypes', 'created', '@context', 'contributors', 'subject', 'bbox', 'creators', 'review_state', 'history', 'connectsWith', 'locations', 'reprPoint'])

In [421]:
dfRoma = pd.DataFrame([jsonRes])
dfRoma

,@context,@type,bbox,connectsWith,contributors,created,creators,description,details,features,...,placeTypes,provenance,references,reprPoint,review_state,rights,subject,title,type,uri
0,"{'created': 'dcterms:created', 'subject': 'dct...",Place,"[12.486137, 41.8917, 12.4862, 41.891775]","[https://pleiades.stoa.org/places/303999556, h...","[{'username': None, 'name': 'DARMC'}, {'userna...",2010-11-10T22:33:36Z,"[{'username': None, 'name': 'L. Quilici'}, {'u...",The capital of the Roman Republic and Empire.,<p>The Barrington Atlas Directory notes: Roma/...,[{'properties': {'location_precision': 'precis...,...,"[urban, settlement, temple-2]",Barrington Atlas: BAtlas 43 B2 Roma,"[{'type': 'seeFurther', 'otherIdentifier': ' '...","[12.4861685, 41.891737500000005]",published,Copyright © The Contributors. Sharing and remi...,"[dare:major=1, dare:feature=major settlement, ...",Roma,FeatureCollection,https://pleiades.stoa.org/places/423025


Zeige verknüpfte Straßen zu dem Hauptort. Hierzu wird für jede Adresse in der Liste `dfRoma[connectsWith]` eine Anfrage gestellt und aus dem resultierenden JSON der Titel ausgelesen. 

In diesem Beispiel werden nur die ersten 20 Links abgefragt. 

In [422]:
connectedStreets = []
for i in (dfRoma['connectsWith'].iloc[0])[:20]:
    res = requests.get(i + '/json')
    try:
        jsonRes = res.json()
        title = jsonRes['title']
        connectedStreets.append(title)
    except:
        pass

In [423]:
connectedStreets

['Via Portuensis?',
 'Via Aemilia Scauri',
 'Via Latina',
 'Via Cornelia',
 'Via Nomentana',
 'Via Ostiensis',
 'Via Ardeatina',
 'Via Tiburtina',
 'Via Collatina',
 'Via Appia',
 'Via Clodia',
 'Via Aurelia',
 'Via Cassia',
 'Via Salaria',
 'Via Flaminia',
 'Via Praenestina',
 'Tiberis (river)',
 'Via Triumphalis',
 'Via Sublacensis']